<a href="https://colab.research.google.com/github/npasuparthi/nagendra-AI_ML/blob/master/Statistical_NLP_Proj_Blog_Analysis_Nagendra_AIMLMar20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import re 
import nltk
from sklearn.feature_extraction.text import CountVectorizer


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# 1. Import the dataset
df = pd.read_csv('/content/drive/MyDrive/NLP/BlogProject/blogtext.csv')

In [8]:
#1. Analyze the dataset
df.describe()

,id,age
count,6.812840e+05,681284.000000
mean,2.397802e+06,23.932326
std,1.247723e+06,7.786009
min,5.114000e+03,13.000000
25%,1.239610e+06,17.000000
50%,2.607577e+06,24.000000
75%,3.525660e+06,26.000000
max,4.337650e+06,48.000000


In [9]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [10]:
df.shape

(681284, 7)

In [11]:
df.nunique(axis=0)

id         19320
gender         2
age           26
topic         40
sign          12
date        2616
text      611652
dtype: int64

In [12]:
# there are no missing entries
df.isna().any()

id        False
gender    False
age       False
topic     False
sign      False
date      False
text      False
dtype: bool

In [13]:
df.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [121]:
# comment out once done
dfSampl = df.sample(30000)

In [122]:
#2. data pre-processing 

In [123]:
dfSampl.loc[(dfSampl['age'] >= 13) & (dfSampl['age'] <=17), 'Age'] = 10
dfSampl.loc[(dfSampl['age'] >= 23) & (dfSampl['age'] <=27), 'Age'] = 20
dfSampl.loc[(dfSampl['age'] >= 33) & (dfSampl['age'] <=47), 'Age'] = 30


In [124]:
dfSampl.describe()

,id,age,Age
count,3.000000e+04,30000.000000,29822.000000
mean,2.395573e+06,23.934733,18.270740
std,1.248408e+06,7.826343,7.061748
min,5.114000e+03,13.000000,10.000000
25%,1.226466e+06,17.000000,10.000000
50%,2.610312e+06,24.000000,20.000000
75%,3.529758e+06,27.000000,20.000000
max,4.334761e+06,48.000000,30.000000


In [125]:
#Remove columns wich are not needed for blog analysis
dfSampl.drop(['id','date', 'age'], axis=1, inplace=True)

In [126]:
dfSampl.describe()

,Age
count,29822.000000
mean,18.270740
std,7.061748
min,10.000000
25%,10.000000
50%,20.000000
75%,20.000000
max,30.000000


In [127]:
import re, string

def clean_str(string):
  """
  String cleaning before vectorization
  """
  try:    
    #string = re.sub(r'^https?:\/\/<>.*[\r\n]*', '', string, flags=re.MULTILINE)
    # retain alphabets, replace everything else with space
    string = re.sub(r"[^A-Za-z]", " ", string)         
    words = string.strip().lower().split()    
    words = [w for w in words if len(w)>=1]
    return " ".join(words)	
  except:
    return ""

In [128]:
dfSampl['text'] = dfSampl['text'].apply(clean_str)
dfSampl.head()
# only alphabets are retained 

,gender,topic,sign,text,Age
464410,male,Student,Taurus,almost done with the thesis i have to change t...,20.0
458824,female,HumanResources,Scorpio,um well for the time being yes but when quiggy...,10.0
455891,female,indUnk,Cancer,urllink you can all jolly well leave my jolly ...,20.0
199983,female,indUnk,Libra,i open my arms to death i search for the big g...,20.0
44950,female,indUnk,Gemini,i signed r up for the summer library reading p...,30.0


In [129]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [130]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [131]:
dfSampl['keywords'] = dfSampl['text'].apply(lambda x : " ".join(word for word in x.split() if word not in stop))


In [132]:
dfSampl.head()

,gender,topic,sign,text,Age,keywords
464410,male,Student,Taurus,almost done with the thesis i have to change t...,20.0,almost done thesis change names thesis reflect...
458824,female,HumanResources,Scorpio,um well for the time being yes but when quiggy...,10.0,um well time yes quiggy comes back going get o...
455891,female,indUnk,Cancer,urllink you can all jolly well leave my jolly ...,20.0,urllink jolly well leave jolly tshirt jolly we...
199983,female,indUnk,Libra,i open my arms to death i search for the big g...,20.0,open arms death search big grim reaper death a...
44950,female,indUnk,Gemini,i signed r up for the summer library reading p...,30.0,signed r summer library reading program done l...


In [133]:
# merge all author columns into a new list  
dfSampl['labels']=dfSampl.apply(lambda col: [col['gender'],str(col['Age']),col['topic'],col['sign']], axis=1)


In [134]:
dfSampl.head()

,gender,topic,sign,text,Age,keywords,labels
464410,male,Student,Taurus,almost done with the thesis i have to change t...,20.0,almost done thesis change names thesis reflect...,"[male, 20.0, Student, Taurus]"
458824,female,HumanResources,Scorpio,um well for the time being yes but when quiggy...,10.0,um well time yes quiggy comes back going get o...,"[female, 10.0, HumanResources, Scorpio]"
455891,female,indUnk,Cancer,urllink you can all jolly well leave my jolly ...,20.0,urllink jolly well leave jolly tshirt jolly we...,"[female, 20.0, indUnk, Cancer]"
199983,female,indUnk,Libra,i open my arms to death i search for the big g...,20.0,open arms death search big grim reaper death a...,"[female, 20.0, indUnk, Libra]"
44950,female,indUnk,Gemini,i signed r up for the summer library reading p...,30.0,signed r summer library reading program done l...,"[female, 30.0, indUnk, Gemini]"


In [135]:
df2 = dfSampl[['keywords', 'labels']]

In [136]:
X = df2['keywords']

In [137]:
Y = df2['labels']

In [138]:
#2. Begin Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(X)


In [139]:
X.shape

(30000, 2000)

In [141]:
df2['labels'].head()

464410              [male, 20.0, Student, Taurus]
458824    [female, 10.0, HumanResources, Scorpio]
455891             [female, 20.0, indUnk, Cancer]
199983              [female, 20.0, indUnk, Libra]
44950              [female, 30.0, indUnk, Gemini]
Name: labels, dtype: object

In [142]:
label_counts=dict()

for labels in df2.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[str(label)]+=1
        else:
            label_counts[str(label)]=1
        
label_counts

{'10.0': 10460,
 '20.0': 14059,
 '30.0': 5303,
 'Accounting': 177,
 'Advertising': 187,
 'Agriculture': 49,
 'Aquarius': 2217,
 'Architecture': 72,
 'Aries': 2824,
 'Arts': 1385,
 'Automotive': 55,
 'Banking': 187,
 'Biotech': 99,
 'BusinessServices': 200,
 'Cancer': 2986,
 'Capricorn': 2227,
 'Chemicals': 172,
 'Communications-Media': 861,
 'Construction': 46,
 'Consulting': 281,
 'Education': 1268,
 'Engineering': 523,
 'Environment': 28,
 'Fashion': 209,
 'Gemini': 2321,
 'Government': 287,
 'HumanResources': 135,
 'Internet': 705,
 'InvestmentBanking': 70,
 'Law': 430,
 'LawEnforcement-Security': 93,
 'Leo': 2427,
 'Libra': 2716,
 'Manufacturing': 98,
 'Maritime': 15,
 'Marketing': 214,
 'Military': 129,
 'Museums-Libraries': 128,
 'Non-Profit': 651,
 'Pisces': 2290,
 'Publishing': 367,
 'RealEstate': 130,
 'Religion': 222,
 'Sagittarius': 2148,
 'Science': 333,
 'Scorpio': 2499,
 'Sports-Recreation': 121,
 'Student': 6880,
 'Taurus': 2769,
 'Technology': 1796,
 'Telecommunications

In [143]:
# Vectorization for Y
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(label_counts.keys()))

In [144]:
Y=binarizer.fit_transform(df2['labels'])


In [145]:
Y.shape
# Number of blogs sampled x number of possible labels (80 of them, only 4 will be active thoough)

(30000, 58)

In [146]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.2)


In [147]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [148]:
model=LogisticRegression(solver='lbfgs')


In [149]:
model=OneVsRestClassifier(model)


In [150]:
model.fit(Xtrain,Ytrain)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [151]:
Ypred=model.predict(Xtest)


In [152]:
Ypred_inversed = binarizer.inverse_transform(Ypred)
y_test_inversed = binarizer.inverse_transform(Ytest)

In [153]:
#5. Print the true vs predicted labels for any 5 entries from the dataset.
for i in range(5):
    print('Text:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        Xtest[i],
        ','.join(y_test_inversed[i]),
        ','.join(Ypred_inversed[i])
    ))

Text:	  (0, 1431)	0.8105487060363953
  (0, 182)	0.2689878359817689
  (0, 1219)	0.22402463877131926
  (0, 647)	0.23632551283053485
  (0, 1258)	0.23296557754475658
  (0, 838)	0.1835017245385825
  (0, 1972)	0.12203545145612044
  (0, 122)	0.24855795662951508
True labels:	20.0,Scorpio,female,indUnk
Predicted labels:	20.0,female


Text:	  (0, 949)	0.27380105422250767
  (0, 587)	0.28562290491084424
  (0, 1300)	0.2784597073079912
  (0, 1293)	0.2609492056568663
  (0, 486)	0.23340502376573757
  (0, 1327)	0.21792128782389547
  (0, 532)	0.22717503859601954
  (0, 924)	0.21949713333531654
  (0, 988)	0.1538225290752623
  (0, 97)	0.19242098089012197
  (0, 149)	0.1985915496261092
  (0, 343)	0.23219686128058384
  (0, 941)	0.1085163677124319
  (0, 1897)	0.2643584702487063
  (0, 101)	0.2504953844922057
  (0, 737)	0.12019035061753393
  (0, 1195)	0.14078774580791126
  (0, 733)	0.11511778441005859
  (0, 1422)	0.11263381022346675
  (0, 931)	0.2052057721156964
  (0, 1574)	0.2844999868015818
  (0, 730)	0.119823

In [154]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def print_evaluation_scores(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: ', f1_score(Ytest, Ypred, average='micro'))
    print('Average precision score: ', average_precision_score(Ytest, Ypred, average='micro'))
    print('Average recall score: ', recall_score(Ytest, Ypred, average='micro'))

In [155]:
print_evaluation_scores(Ytest, Ypred)


Accuracy score:  0.0018333333333333333
F1 score:  0.40063287562863764
Average precision score:  0.23241713979327694
Average recall score:  0.29541666666666666
